This Python program takes GWAS data from ../../data/2018_05_29_PheWAS_and_GWAS/Zhang et al PTB 10000/pre_top10000.tab
and PheWAS data from ../../data/2018_05_29_PheWAS_and_GWAS/PheWAS Catalog/phewas-catalog.csv and finds the intersection of the SNPs, and stored the results to the stored in the ../../results/06_08_2018/combined folder. It then splits the data into individual files based on the SNP (stored in the ../../results/06_08_2018/snp folder) and calls an R program to create a scatter plot for each SNP by taking the -log(p_value).

In [1]:
# Import pandas for data combining
import pandas as pd

# For csv file writing
import csv

In [2]:
# Open the GWAS catalog as a dataframe.
gwas_df = pd.read_table("../../data/2018_05_29_PheWAS_and_GWAS/Zhang et al PTB 10000/pre_top10000.tab")

# Get column of GWAS containing only the SNPs and convert to dataframe
gwas_snps_df = pd.DataFrame(gwas_df['snp'])

# Open the PheWAS catalog as a dataframe.
phewas_df = pd.read_csv("../../data/2018_05_29_PheWAS_and_GWAS/PheWAS Catalog/phewas-catalog.csv")


d:\program files (x86)\python 3.6.5\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Use inner join to combine the GWAS file with PheWAS file
combined_df = pd.merge(gwas_snps_df, phewas_df, how='inner', on='snp')

# Write the result to a csv file
combined_df.to_csv("../../results/2018_06_08/combined/Combined_GWAS_PheWAS.csv", sep = '\t', index = False)

In [4]:
# Check to see which GWAS SNPs were not in the PheWAS
missing_snps_df = gwas_snps_df[~gwas_snps_df.snp.isin(phewas_df.snp)]

# Print out those SNPs that are missing
missing_snps_df.to_csv("../../results/2018_06_08/combined/missing_snps.csv", sep = '\t', index = False)

In [5]:
# Count number of repetitions of each SNP and print to csv file.
repetitions_df = combined_df['snp'].value_counts()
repetitions_df.to_csv("../../results/2018_06_08/combined/repetitions.csv", sep = '\t')

In [6]:
# Double Check results with sets
phewas_set = set(phewas_df['snp'])
gwas_set = set(gwas_df['snp'])
intersection_set = phewas_set.intersection(gwas_set)
with open('../../results/2018_06_08/combined/set_output.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter = '\t')
    csv_writer.writerow(list(intersection_set))

In [7]:
# Split up the combined dataframe into separate ones by SNP
grouped_snps_df = combined_df.groupby(by = "snp")

# Within each SNP, sort by p-value, then odds, then phewas_string. Use mergesort starting from phewas_string, since
# mergesort preserves relative order.
for snp, group in grouped_snps_df:
    sorted_phewas_df = group.sort_values(by = 'phewas_string', kind = 'mergesort')
    sorted_odds_df = sorted_phewas_df.sort_values(by = 'odds_ratio', ascending = False, kind = 'mergesort')
    sorted_p_df = sorted_odds_df.sort_values(by = 'p', kind = 'mergesort')
    # Write the result to a csv file
    sorted_p_df.to_csv('../../results/2018_06_08/snps/' + str(snp) +'.csv', index = False ,sep = '\t')